In [1]:
import numpy as np
import pandas as pd

# from keras.models import Sequential
# from keras.layers import Dense
# from keras.callbacks import EarlyStopping

# from sklearn.preprocessing import StandardScaler
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

import matplotlib
matplotlib.use('nbagg')
import matplotlib.pyplot as plt

In [2]:
def plot_bar(quantidade, label):
    
    ind = np.arange(len(quantidade))
    width = 0.8
    
    fig, ax = plt.subplots()
    rects1 = ax.bar(ind, quantidade, width, color='b')
    
    ax.set_ylabel('Quantidade')
    ax.set_title('Gráfico de barras')
    ax.set_xticks(ind)
    ax.set_xticklabels(label)
    
    ax.legend()
    plt.show()

# plot_bar([5,3,4], ["x1","x2","x3"])

In [3]:
# read songs
data = pd.read_csv('data.csv'#, nrows=1000000
                  )

MemoryError: 

In [ ]:
# read train
train = pd.read_csv('train.csv'#, sep=',', nrows=1000
                   ) # nrows = ler as n primeiras linhas do arquivo

In [ ]:
#read members
member = pd.read_csv('members.csv')

In [ ]:
# read songs
song = pd.read_csv('songs.csv'#, nrows=1000000
                  )

In [ ]:
song_extra = pd.read_csv('song_extra_info.csv'#, nrows=1000000
                  )

In [ ]:
test = pd.read_csv('test.csv', index_col=0)

In [ ]:
columns_member = member.columns.tolist()
columns_train = train.columns.tolist()
columns_song = song.columns.tolist()
columns_song_extra = song_extra.columns.tolist()
print (columns_member)
print (columns_train)
print (columns_song)
print (columns_song_extra)

In [ ]:

tt = train.merge(member[columns_member], on='msno', how='left')
tt = tt.merge(song[columns_song], on='song_id', how='left')
tt = tt.merge(song_extra[columns_song_extra], on='song_id', how='left')
tt

In [ ]:
tttest = test.merge(member[columns_member], on='msno', how='left')
tttest = tttest.merge(song[columns_song], on='song_id', how='left')
tttest = tttest.merge(song_extra[columns_song_extra], on='song_id', how='left')
tttest

### Ananlise da quantidade de dados nulos por coluna

In [ ]:
true = []
false = []
labels = []

for i in tt.columns.tolist():
    vazio = tt[tt[i].isnull() | tt[i].isin(['null'])]
    true.append(len(tt)-len(vazio))
    false.append(len(vazio))
    labels.append(i)
    print (len(vazio), i)

ind = np.arange(len(true)) 
width = 0.75 

fig, ax = plt.subplots(figsize=(45, 5))

p1 = ax.bar(ind, tuple(true), width, color='blue')
p2 = ax.bar(ind, tuple(false), width,
             bottom=true, color='red')

ax.set_ylabel('Scores')
ax.set_title('Scores by group')
ax.set_xticks(ind)
ax.set_xticklabels(labels)
ax.legend((p1[0], p2[0]), ('Válido', 'nan, null'))

plt.show()

In [ ]:
columns_tt = data.columns.tolist()
for i in columns_tt:
    cc = list(data[i].unique())
    print (i, "==>" , len(cc), type(cc[0]) , "\n")

In [ ]:
columns_dftrain = [
 'target',
 'source_system_tab',
 'source_screen_name',
 'source_type',
 'city',
 'bd',
 'registered_via',
 'registration_init_time',
 'expiration_date',
 'song_length',
 'language',
 'isrc']

dftrain = tt.reindex(columns=columns_dftrain)
dftrain

In [ ]:
for i in data.columns.tolist():
    value = data[np.logical_or(data[i].isnull(), data[i].isin(['null']))]
    print (len(value), i)

# for i in dftrain.columns.tolist():
#     dftrain[i].replace([np.nan, "null"], 0, inplace=True)
    
# for i in dftest.columns.tolist():
#     value = dftest[np.logical_not(np.logical_or(dftest[i].isnull(), dftest[i].isin(['null'])))]
#     print (len(value), i)

In [5]:
data['isrc'].replace([np.nan, "null"], 0, inplace=True)

In [ ]:
# str columns
str_columns = ['source_system_tab', 'source_screen_name', 'source_type']
for i in str_columns:
    print (i)
    dummies = pd.get_dummies(dftrain[i], prefix=i)
    dftrain = pd.concat([dftrain,dummies], axis=1)
dftrain
# dummies = pd.get_dummies(dftrain['gender'], prefix='gender')
# dftttrain = pd.concat([dftrain, dummies], axis=1)
# # dftrain.join(dummies)
# dftttrain

### Removendo colunas que foram feito dummies do dataframe

In [ ]:
dftrain.drop(str_columns, axis=1, inplace=True)
dftrain

### Tratando a coluna isrc

In [ ]:
isrc_values = pd.Series(dftrain.isrc.values)
dftrain.drop(['isrc'], axis=1, inplace=True)
# isrc_column = dftrain.reindex(columns=['isrc'])

# isrc_column['cc'] = isrc_values.str.slice(0,2)  # Country Code column
# isrc_column['xxx'] = isrc_values.str.slice(2,5) # IRSC Issuer
dftrain['isrc'] = isrc_values.str.slice(5,7).astype(float)  # IRSC issue date
dftrain

ajustes na coluna de ano

In [ ]:
dftrain.loc[dftrain['isrc'] > 17, 'isrc'] += 1900  # 1900's songs
dftrain.loc[dftrain['isrc'] < 18, 'isrc'] += 2000  # 2000's songs

dftrain

treinamento

In [ ]:
dftrain.to_csv('data.csv.gz', compression = 'gzip', index=False)

In [6]:
X = data.drop(['target'], axis=1)
y = data.target
X

,city,bd,registered_via,registration_init_time,expiration_date,song_length,language,source_system_tab_0,source_system_tab_discover,source_system_tab_explore,...,source_type_local-library,source_type_local-playlist,source_type_my-daily-playlist,source_type_online-playlist,source_type_radio,source_type_song,source_type_song-based-playlist,source_type_top-hits-for-artist,source_type_topic-article-playlist,isrc
0,1,0,7,20120102,20171005,206471.0,52.0,0,0,1,...,0,0,0,1,0,0,0,0,0,2016.0
1,13,24,9,20110525,20170911,284584.0,52.0,0,0,0,...,0,1,0,0,0,0,0,0,0,1999.0
2,13,24,9,20110525,20170911,225396.0,52.0,0,0,0,...,0,1,0,0,0,0,0,0,0,2006.0
3,13,24,9,20110525,20170911,255512.0,-1.0,0,0,0,...,0,1,0,0,0,0,0,0,0,2010.0
4,1,0,7,20120102,20171005,187802.0,52.0,0,0,1,...,0,0,0,1,0,0,0,0,0,2016.0
5,1,0,7,20120102,20171005,247803.0,52.0,0,0,1,...,0,0,0,1,0,0,0,0,0,2016.0
6,13,24,9,20110525,20170911,229982.0,31.0,0,0,0,...,0,1,0,0,0,0,0,0,0,2014.0
7,1,0,7,20120102,20171005,181115.0,52.0,0,0,1,...,0,0,0,1,0,0,0,0,0,2016.0
8,15,26,9,20111107,20180304,278964.0,52.0,0,0,0,...,1,0,0,0,0,0,0,0,0,2007.0
9,15,26,9,20111107,20180304,257369.0,52.0,0,0,0,...,1,0,0,0,0,0,0,0,0,2013.0


In [16]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression(normalize=True)
reg.fit(X[50000:],y[50000:])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=True)

In [27]:
a = reg.predict(X[:50000])
print (list(filter(lambda x : x < 0.15, a)))
a = list(map(lambda x: 1 if x > 0.8 else 0, a))
b = pd.Series(a, y[:50000])
b

[0.134613037109375, 0.13417816162109375]


target
1    0
1    0
1    0
1    1
1    0
1    0
1    1
1    0
1    1
1    1
1    1
1    1
0    0
1    0
1    1
1    0
1    1
1    1
0    0
1    1
1    1
1    1
1    0
1    0
0    0
0    0
0    0
1    1
1    1
1    0
    ..
1    1
1    0
1    1
1    0
1    1
1    1
1    1
1    1
0    0
1    1
1    1
1    0
1    0
1    0
0    0
0    0
0    0
1    0
0    0
1    0
0    0
0    0
0    0
1    0
0    0
0    0
1    0
1    0
1    0
0    0
Length: 50000, dtype: int64

## Teste

In [ ]:
columns_dftest = [
 'source_system_tab',
 'source_screen_name',
 'source_type',
 'city',
 'bd',
 'registered_via',
 'registration_init_time',
 'expiration_date',
 'song_length',
 'language',
 'isrc']

dftest = tttest.reindex(columns=columns_dftest)
dftest

#### tratando os valores nulos

In [ ]:
# for i in dftest.columns.tolist():
#     value = dftest[np.logical_not(np.logical_or(dftest[i].isnull(), dftest[i].isin(['null'])))]
#     print (len(value), i)

for i in dftest.columns.tolist():
    dftest[i].replace([np.nan, "null"], 0, inplace=True)
    
# for i in dftest.columns.tolist():
#     value = dftest[np.logical_not(np.logical_or(dftest[i].isnull(), dftest[i].isin(['null'])))]
#     print (len(value), i)
    

#### tratando colunas categoricas

In [ ]:
str_columns = ['source_system_tab', 'source_screen_name', 'source_type']
for i in str_columns:
    print (i)
    dummies = pd.get_dummies(dftest[i], prefix=i)
    dftest = pd.concat([dftest,dummies], axis=1)
dftest

In [ ]:
dftest.drop(str_columns, axis=1, inplace=True)
dftest

#### tratando coluna isrc

In [ ]:
isrc_values_test = pd.Series(dftest.isrc.values)
dftest.drop(['isrc'], axis=1, inplace=True)
# isrc_column = dftrain.reindex(columns=['isrc'])

# isrc_column['cc'] = isrc_values.str.slice(0,2)  # Country Code column
# isrc_column['xxx'] = isrc_values.str.slice(2,5) # IRSC Issuer
dftest['isrc'] = isrc_values_test.str.slice(5,7).astype(float)  # IRSC issue date

In [ ]:
dftest.loc[dftest['isrc'] > 17, 'isrc'] += 1900  # 1900's songs
dftest.loc[dftest['isrc'] < 18, 'isrc'] += 2000  # 2000's songs

dftest

In [ ]:
reg.predict(dftest)

In [ ]:
columns_isrc = isrc_column.columns.tolist()
for i in columns_isrc:
    cc = list(isrc_column[i].unique())
    print (i, "==>" , len(cc), type(cc[0]) , "\n")

junstando com a base de trainamento

In [ ]:
dftrain.drop(['isrc'], axis=1, inplace=True) # removendo a coluna isrc de dftrain
dftrain = pd.concat([dftrain,isrc_column], axis=1)
dftrain

In [ ]:
columns = tt.columns.tolist()
# print (columns)
dd = tt.reindex(columns=columns)
# dd.plot.bar(x="XA", y="BA")
# print (len(dd.msno.unique()))
# print (len(dd.song_id.unique()))
datas = []
labels = []
for column in columns:
    datas.append(len(dd[column].unique()))
    labels.append(column)
    
print (datas, labels)
print ("\n", dd.genre_ids.unique())
for i in dd.genre_ids.unique():
    print (len(dd[dd.genre_ids.isin([i])]), i)
plot_bar(datas[2:], labels[2:0])

In [ ]:
def source_system_tab(data_frame):
    """
    recebe o data_frame da base de dados e retorna o data frame retirando as 
    linhas que tem valores nulos  na coluna source_system_tab
    """
    df = data_frame[pd.notnull(data_frame.source_system_tab)]
    df = df[~df.source_system_tab.isin(["null"])]
    return df
for i in df.source_system_tab.unique():
    print (i, len(df[df.source_system_tab.isin([i])]))

In [ ]:
tt.describe()

In [ ]:
columns = data_set.columns.tolist()[:-1]
data_set.drop_duplicates(subset=columns, keep='first', inplace=True) 

In [ ]:
data_set.describe()

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb

from sklearn.neural_network import MLPRegressor

print('Loading data...')
data_path = './'
train = pd.read_csv(data_path + 'train.csv', nrows=50000, dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                  'source_screen_name' : 'category',
                                                  'source_type' : 'category',
                                                  'target' : np.uint8,
                                                  'song_id' : 'category'})
test = pd.read_csv(data_path + 'test.csv', nrows=50000, dtype={'msno' : 'category',
                                                'source_system_tab' : 'category',
                                                'source_screen_name' : 'category',
                                                'source_type' : 'category',
                                                'song_id' : 'category'})
songs = pd.read_csv(data_path + 'songs.csv', nrows=50000, dtype={'genre_ids': 'category',
                                                  'language' : 'category',
                                                  'artist_name' : 'category',
                                                  'composer' : 'category',
                                                  'lyricist' : 'category',
                                                  'song_id' : 'category'})
members = pd.read_csv(data_path + 'members.csv',dtype={'city' : 'category',
                                                      'bd' : np.uint8,
                                                      'gender' : 'category',
                                                      'registered_via' : 'category'})
songs_extra = pd.read_csv(data_path + 'song_extra_info.csv', nrows=50000)

print('Data preprocessing...')
song_cols = ['song_id', 'artist_name', 'genre_ids', 'song_length', 'language']
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration_date'] = members['registration_init_time'].apply(lambda x: int(str(x)[6:8]))

members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration_date'] = members['expiration_date'].apply(lambda x: int(str(x)[6:8]))
members = members.drop(['registration_init_time'], axis=1)

def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan
        
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

train = train.merge(members, on='msno', how='left')
test = test.merge(members, on='msno', how='left')

train = train.merge(songs_extra, on = 'song_id', how = 'left')
test = test.merge(songs_extra, on = 'song_id', how = 'left')

import gc
del members, songs; gc.collect();

for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')

X = train.drop(['target'], axis=1)
y = train['target'].values

print (X)
X_test = test.drop(['id'], axis=1)
ids = test['id'].values


mlp = MLPRegressor(hidden_layer_sizes=(3), activation='relu', verbose=True)
mlp.fit(X, y)

p_test = mlp.predict(X_test)

"""
del train, test; gc.collect();

d_train = lgb.Dataset(X, y)
watchlist = [d_train]

#Those parameters are almost out of hat, so feel free to play with them. I can tell
#you, that if you do it right, you will get better results for sure ;)
print('Training LGBM model...')
params = {}
params['learning_rate'] = 0.2
params['application'] = 'binary'
params['max_depth'] = 8
params['num_leaves'] = 2**8
params['verbosity'] = 0
params['metric'] = 'auc'

print (d_train)

model = lgb.train(params, train_set=d_train, num_boost_round=50, valid_sets=watchlist, \
verbose_eval=5)

print('Making predictions and saving them...')
p_test = model.predict(X_test)
"""
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
subm.to_csv('submissio.csv.gz', compression = 'gzip', index=False, float_format = '%.5f')
print('Done!')

In [ ]:
x = np.random.randn(1000, 3)
print (x)
# fig, axes = plt.subplots()

colors = ['red', 'blue', 'green']
N = 7
menMeans = [20, 35, 30, 35, 27,5,2]
womenMeans = [25, 32, 34, 20, 25,15,3]
menStd = (2, 3, 4, 1, 2)
womenStd = (3, 5, 2, 3, 3)
ind = np.arange(N)    # the x locations for the groups
width = 0.75       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, tuple(menMeans), width, color='#d62728'#, yerr=menStd
            )
p2 = plt.bar(ind, tuple(womenMeans), width,
             bottom=menMeans#, yerr=womenStd
            )

plt.ylabel('Scores')
plt.title('Scores by group and gender')
plt.xticks(ind, ['G1', 'G2', 'G3', 'G4', 'G5'])
plt.yticks(np.arange(0, 81, 10))
plt.legend((p1[0], p2[0]), ('Men', 'Women'))

plt.show()